 ## Первый раздел.
 ### Импорт библиотек, подключение CUDA devices

In [33]:
!pip install numba
!pip install py-cpuinfo
from datetime import datetime
from tabulate import tabulate
from datetime import datetime
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from tabulate import tabulate

import matplotlib.pyplot as plt
import numpy as np
import cpuinfo
import math
import matplotlib.pyplot as plt
import numba
import pandas as pd
import time

### *Проверяем, какое оборудование у нас подключенно.*:

In [34]:
print('----------------------------------------------------------------------')
print('CPU, выделенный на сервере Colaboratory:', cpuinfo.get_cpu_info()['brand_raw'])
print('GPU, выделенный на сервере Colaboratory:', cuda.get_current_device())
print('----------------------------------------------------------------------')
numba.cuda.detect()

----------------------------------------------------------------------
CPU, выделенный на сервере Colaboratory: Intel(R) Xeon(R) CPU @ 2.30GHz
GPU, выделенный на сервере Colaboratory: <CUDA device 0 'b'Tesla K80''>
----------------------------------------------------------------------
Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

### Второй раздел.

Функции вычисления на CPU и GPU <br/>

In [35]:
TPB = 16
number_iterations = 16
iterations = 10

Arr_Pi_CPU = []
Arr_Pi_GPU = []

Arr_timeCPU = []
Arr_timeGPU = []

CPU_time =[]
GPU_time =[]

In [36]:
def CPU_Pi_Calc(number_iterations, N):
    res = np.zeros(number_iterations)
    for i in range(number_iterations):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

In [37]:
@cuda.jit 
def GPU_Pi_Calc(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  h = 0
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)

In [38]:
def DeviceHost(N):

  rng_states = create_xoroshiro128p_states(N, seed=1)
  res = [0]

  dev_res = cuda.to_device(res)
  GPU_Pi_Calc[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()
  pi = 4 * tmp_[0] / N
  return pi

In [39]:
def main(iterations , N):
  
  for i in range(iterations):
    time_CPU = time.time()
    pi_CPU = CPU_Pi_Calc(number_iterations, N).mean()   
    time_CPU = (time.time() - time_CPU) 
    Arr_Pi_CPU.append(pi_CPU)
    Arr_timeCPU.append(time_CPU)

    time_GPU = time.time()
    pi_GPU = DeviceHost(N)
    N = N + 10000  
    time_GPU = (time.time() - time_GPU)
    Arr_Pi_GPU.append(pi_GPU)
    Arr_timeGPU.append(time_GPU)

    print(f"Номер итерации: {i+1}")
    print('----------------------------------------------------------------------')
    print(f"Время вычисления на CPU: {round(time_CPU,2)} second.")
    print(f"Число Pi на CPU: {pi_CPU}")
    print('----------------------------------------------------------------------')
    print(f"Время вычисления на GPU: {round(time_GPU,2)} second.")
    print(f"Число Pi на GPU: {pi_GPU}")
    print('----------------------------------------------------------------------')
    print("                                       ")


### Третий раздел.

*Выходные данные* <br/>

In [40]:
if __name__ == "__main__":
  print(f"Количество итераций: {iterations}")
  main(iterations, 10000)


Количество итераций: 10
Номер итерации: 1
----------------------------------------------------------------------
Время вычисления на CPU: 0.44 second.
Число Pi на CPU: 3.1379
----------------------------------------------------------------------
Время вычисления на GPU: 0.21 second.
Число Pi на GPU: 3.1552
----------------------------------------------------------------------
                                       
Номер итерации: 2
----------------------------------------------------------------------
Время вычисления на CPU: 0.82 second.
Число Pi на CPU: 3.1420375
----------------------------------------------------------------------
Время вычисления на GPU: 0.01 second.
Число Pi на GPU: 3.1578
----------------------------------------------------------------------
                                       
Номер итерации: 3
----------------------------------------------------------------------
Время вычисления на CPU: 1.2 second.
Число Pi на CPU: 3.1395916666666666
---------------------

In [41]:
for i in range(10):
  CPU_time.append(str(round(Arr_timeCPU[i], 2)))
  GPU_time.append(str(round(Arr_timeGPU[i], 2)))


timing_df = pd.DataFrame({'Device':['CPU', 'GPU', 'CPU' ,'GPU', 'CPU', 'GPU', 'CPU' ,'GPU', 'CPU' ,'GPU',
                                    'CPU', 'GPU', 'CPU' ,'GPU', 'CPU', 'GPU', 'CPU' ,'GPU', 'CPU' ,'GPU'],
                          'Time(sec)':[CPU_time[0], GPU_time[0], CPU_time[1], GPU_time[1],
                                       CPU_time[2], GPU_time[2], CPU_time[3], GPU_time[3],
                                       CPU_time[4], GPU_time[4], CPU_time[5], GPU_time[5],
                                       CPU_time[6], GPU_time[6], CPU_time[7], GPU_time[7],
                                       CPU_time[8], GPU_time[8], CPU_time[9], GPU_time[9]],
                          'Number_Pi':[f"{Arr_Pi_CPU[0]}", f"{Arr_Pi_GPU[0]}",f"{Arr_Pi_CPU[1]}", f"{Arr_Pi_GPU[1]}",
                                       f"{Arr_Pi_CPU[2]}", f"{Arr_Pi_GPU[2]}",f"{Arr_Pi_CPU[3]}", f"{Arr_Pi_GPU[3]}",
                                       f"{Arr_Pi_CPU[4]}", f"{Arr_Pi_GPU[4]}",f"{Arr_Pi_CPU[5]}", f"{Arr_Pi_GPU[5]}",
                                       f"{Arr_Pi_CPU[6]}", f"{Arr_Pi_GPU[6]}",f"{Arr_Pi_CPU[7]}", f"{Arr_Pi_GPU[7]}",
                                       f"{Arr_Pi_CPU[8]}", f"{Arr_Pi_GPU[8]}",f"{Arr_Pi_CPU[9]}", f"{Arr_Pi_GPU[9]}"]})
                                         
timing_df




,Device,Time(sec),Number_Pi
0,CPU,0.44,3.1379
1,GPU,0.21,3.1552
2,CPU,0.82,3.1420375
3,GPU,0.01,3.1578
4,CPU,1.2,3.1395916666666666
5,GPU,0.01,3.1508
6,CPU,1.62,3.14519375
7,GPU,0.01,3.1471
8,CPU,1.97,3.139025
9,GPU,0.02,3.14304
